<a href="https://colab.research.google.com/github/Fischchen/KatzenErkennung/blob/main/KatzenAbwehr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare dataset
## Prepares dataset by renaming, sorting and alterating images


In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d andrewmvd/dog-and-cat-detection
!unzip -q dog-and-cat-detection.zip

Saving kaggle.json to kaggle.json
 99% 1.02G/1.03G [00:15<00:00, 73.8MB/s]
100% 1.03G/1.03G [00:15<00:00, 69.7MB/s]


In [2]:
import xml.etree.ElementTree as ET
import os


all_xmls = os.listdir("./annotations")
imagePath="./images"
annotationsPath="./annotations"
for xml_file in all_xmls:
  image=""
  imageType=""
  if(True):
    content = open("./annotations/"+xml_file,"r")
    tree = ET.parse(content)
    root = tree.getroot()
    for name in root.iter("filename"):
      #print(name.text)
      image=name.text
    for image_Type in root.iter("name"):
      #print(image_Type.text)
      imageType = image_Type.text
    if (imageType=="dog"):
      oldname = image
      newname = oldname.replace("Cats","Dogs")
      try:
        os.rename(imagePath+"/"+image,imagePath+"/"+newname)
      except:
        pass
      oldXMLName = xml_file
      newXMLName = oldXMLName.replace("Cats","Dogs")
      content.close()
      os.rename(annotationsPath+"/"+oldXMLName,annotationsPath+"/"+newXMLName)

    



In [3]:
from PIL import Image
from PIL import ImageFile
from tqdm import tqdm_notebook
import os
import threading

ImageFile.LOAD_TRUNCATED_IMAGES = True
all_images = os.listdir("./images/")
pbar = tqdm(total=len(all_images))
def convertImage(image):
    pbar.set_description(f"Processing {image}")
    imageToProcces = Image.open("./images/"+image)
    os.remove("./images/"+image)
    newImage = imageToProcces.convert("RGBA")
    newImage.save("./images/"+image.replace("jpg","png"))
    pbar.update(1)

for image in all_images:
  x = threading.Thread(target=convertImage,args=(image,))
  x.start()

print(f"\nConverted: {len(all_images)} images to png")


Processing Dogs_Test679.png:  99%|█████████▊| 3635/3686 [04:34<00:03, 16.81it/s] 


Converted: 3686 images to png


Processing Cats_Test2544.png:  99%|█████████▊| 3636/3686 [04:34<00:02, 16.81it/s]

In [4]:
from scipy.io import savemat
import xml.etree.ElementTree as ET
import sys



def extract_mat_contents(annot_directory):

    content = open("./annotations/"+annot_directory,"r")
    tree = ET.parse(content)
    root = tree.getroot()

    for name in root.iter("xmin"):
        x1 = int(name.text)
    for name in root.iter("ymin"):
        y1 = int(name.text)
    for name in root.iter("xmax"):
        x2 = int(name.text)
    for name in root.iter("ymax"):
        y2 = int(name.text)
    # os.close(content)
    content.close()
    # Get the height and width for our image

    # Get the bounding box co-ordinates
    
    save_touple = (x1, y1, x2, y2)
    toSave = {"box_coord": save_touple}
    os.remove("./annotations/"+annot_directory)
    savemat("./annotations/"+annot_directory.replace(".xml",".mat"),toSave)

    # Return the extracted attributes
    return


all_files = os.listdir("./annotations")
for file in all_files:
  extract_mat_contents(file)


Processing Cats_Test2544.png: 100%|█████████▉| 3683/3686 [04:36<00:00, 35.19it/s]

In [5]:
import os
import shutil
import random


def distribute_annotations_train_validation_split(validation_size=0.2):

    all_images = os.listdir("./annotations")
   #print(all_images)
    random.shuffle(all_images)

    all_dogs = list(filter(lambda image: "Dogs" in image, all_images))
    all_cats = list(filter(lambda image: "Cats" in image, all_images))

    index_to_split = int(len(all_dogs) - len(all_dogs) * validation_size)
    training_dogs = all_dogs[:index_to_split]
    validation_dogs = all_dogs[index_to_split:]
    training_cats = all_cats[:index_to_split]
    validation_cats = all_cats[index_to_split:]
    os.makedirs("./sorted_annotations",exist_ok=True)
    shutil.rmtree("./sorted_annotations")
    os.makedirs("./sorted_annotations/train/dogs/", exist_ok=True)
    os.makedirs("./sorted_annotations/train/cats/", exist_ok=True)
    os.makedirs("./sorted_annotations/validation/dogs/", exist_ok=True)
    os.makedirs("./sorted_annotations/validation/cats/", exist_ok=True)

    copy_annotations_to_dir(training_dogs, "./sorted_annotations/train/dogs")
    copy_annotations_to_dir(validation_dogs, "./sorted_annotations/validation/dogs")
    copy_annotations_to_dir(training_cats, "./sorted_annotations/train/cats")
    copy_annotations_to_dir(validation_cats, "./sorted_annotations/validation/cats")

def distribute_images_train_validation_split(validation_size=0.2):

    all_images = os.listdir("./images")

    random.shuffle(all_images)

    all_dogs = list(filter(lambda image: "Dogs" in image, all_images))
    all_cats = list(filter(lambda image: "Cats" in image, all_images))

    index_to_split = int(len(all_dogs) - len(all_dogs) * validation_size)
    print(f"dogs:{len(all_dogs)} \ncats: {len(all_cats)}")
    training_dogs = all_dogs[:index_to_split]
    validation_dogs = all_dogs[index_to_split:]
    training_cats = all_cats[:index_to_split]
    validation_cats = all_cats[index_to_split:]
    os.makedirs("./sorted_images",exist_ok=True)
    shutil.rmtree("./sorted_images")
    os.makedirs("./sorted_images/train/dogs/", exist_ok=True)
    os.makedirs("./sorted_images/train/cats/", exist_ok=True)
    os.makedirs("./sorted_images/validation/dogs/", exist_ok=True)
    os.makedirs("./sorted_images/validation/cats/", exist_ok=True)

    copy_images_to_dir(training_dogs, "./sorted_images/train/dogs")
    copy_images_to_dir(validation_dogs, "./sorted_images/validation/dogs")
    copy_images_to_dir(training_cats, "./sorted_images/train/cats")
    copy_images_to_dir(validation_cats, "./sorted_images/validation/cats")


def copy_images_to_dir(images_to_copy, destination):
    for image in images_to_copy:
        #print(f"copying: {image} to : {destination}")
        shutil.copyfile(f"./images/{image}", f"{destination}/{image}")
def copy_annotations_to_dir(images_to_copy, destination):
    for image in images_to_copy:
        #print(f"copying: {image} to : {destination}")
        shutil.copyfile(f"./annotations/{image}", f"{destination}/{image}")

if(__name__=="__main__"):
  distribute_images_train_validation_split(0.2)
  distribute_annotations_train_validation_split(0.2)
  
  #print("abc")

dogs:2498 
cats: 1188
dogs:2498 
cats: 1188


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import imgaug.augmenters as iaa
import imgaug
from scipy.io import loadmat
import cv2
import imageio
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import re
from tqdm import tqdm_notebook



def extract_mat_contents(annot_directory, image_dir):
        
        # Create MAT Parser
        mat = loadmat(annot_directory,)

        # Get the height and width for our image
        height, width = cv2.imread(image_dir).shape[:2]

        # Get the bounding box co-ordinates
        x1, y2, y1, x2 = tuple(map(tuple, mat["box_coord"]))[0]

        # We Split the image Directory passed in the method and choose the index
        # Of the Folders name which is the same as it"s class
        class_name = image_dir.split("/")[2]

        filename = "/".join(image_dir.split("/")[-2:])

        # Return the extracted attributes
        return filename,  width, height, class_name, x1,y1,x2,y2

def bounding_boxes_to_df(bounding_boxes_object):

    # Convert Bounding Boxes Object to Array
    bounding_boxes_array = bounding_boxes_object.to_xyxy_array()
    
    # Convert the array into DataFrame
    df_bounding_boxes = pd.DataFrame(bounding_boxes_array, 
                                     columns=["xmin", "ymin", "xmax", "ymax"])
    
    # Return the DataFrame
    return df_bounding_boxes
def mat_to_csv(annot_directory, image_directory, classes_folders):

  # List containing all our attributes regarding each image
  mat_list = []

  # We loop our each class and its labels one by one to preprocess and augment 
  for class_folder in classes_folders:

    # Set our images and annotations directory
    image_dir = os.path.join(image_directory, class_folder)
    annot_dir = os.path.join(annot_directory, class_folder) 

    # Get each file in the image and annotation directory
    mat_files = sorted(os.listdir(annot_dir))
    img_files = sorted(os.listdir(image_dir))

    # Loop over each of the image and its label
    for mat, image_file in zip(mat_files, img_files):
      
      # Full mat path
      mat_path = os.path.join(annot_dir, mat)

      # Full path Image
      img_path = os.path.join(image_dir, image_file)

      # Get Attributes for each image 
      value = extract_mat_contents(mat_path, img_path)

      # Append the attributes to the mat_list
      mat_list.append(value)

  # Columns for Pandas DataFrame
  column_name = ["filename", "width", "height", "class", "xmin", "ymin", 
                 "xmax", "ymax"]

  # Create the DataFrame from mat_list
  mat_df = pd.DataFrame(mat_list, columns=column_name)

  # Return the dataframe
  return mat_df
# Function to convert MAT files to CSV


# The Classes we will use for our training
classes_list = sorted(["dogs",  "cats"])


# Set our images and annotations directory
image_directory = "sorted_images/train"
annot_directory = "sorted_annotations/train"

# Run the function to convert all the MAT files to a Pandas DataFrame
labels_df = mat_to_csv(annot_directory, image_directory, classes_list)

# Saving the Pandas DataFrame as CSV File
labels_df.to_csv(("labels.csv"), index=None)

# Function to convert bounding box image into DataFrame 
# Define all the Augmentations you want to apply to your dataset
# We"re setting random `n` agumentations to 2. 
image_augmentations = iaa.SomeOf( 2,
    [                                 
    # Scale the Images
    iaa.Affine(scale=(0.5, 1.5)),

    # Rotate the Images
    iaa.Affine(rotate=(-60, 60)),

    # Shift the Image
    iaa.Affine(translate_percent={"x":(-0.3, 0.3),"y":(-0.3, 0.3)}),

    # Flip the Image
    iaa.Fliplr(1),

    # Increase or decrease the brightness
    iaa.Multiply((0.5, 1.5)),

    # Add Gaussian Blur
    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    
    # Add Gaussian Noise
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))

])
def image_aug(df, images_path, aug_images_path, augmentor, multiple = 3):
    all_Image=os.listdir(images_path)
    pbar = tqdm(total=len(all_images)*multiple,position=0, leave=True)
    # Fill this DataFrame with image attributes
    augmentations_df = pd.DataFrame(
        columns=["filename","width","height","class", "xmin", "ymin", "xmax",
                 "ymax"])
    
    # Group the data by filenames
    grouped_df = df.groupby("filename")

    # Create the directory for all augmentated images
    if not os.path.exists(aug_images_path):
      os.mkdir(aug_images_path)

    # Create directories for each class of augmentated images
    for folder in df["class"].unique():
      if not os.path.exists(os.path.join(aug_images_path, folder)):
        os.mkdir(os.path.join(aug_images_path, folder))
    
    for i in range(multiple):
      
      # Post Fix we add to the each different augmentation of one image
      image_postfix = str(i)
      
      # Loop to perform the augmentations
      for filename in df["filename"].unique():
        pbar.update(1)
        pbar.set_description(f"Processing {filename}")
        augmented_path = os.path.join(aug_images_path, filename)+image_postfix+".png"

        # Take one image at a time with its information
        single_image = grouped_df.get_group(filename)
        single_image = single_image.reset_index()
        single_image = single_image.drop(["index"], axis=1)   
        
        # Read the image
        image = imageio.imread(os.path.join(images_path, filename))

        # Get bounding box
        bounding_box_array = single_image.drop(["filename", "width", "height",
                                                "class"], axis=1).values

        # Give the bounding box to imgaug library
        bounding_box = BoundingBoxesOnImage.from_xyxy_array(bounding_box_array, 
                                                            shape=image.shape)

        # Perform random 2 Augmentations
        image_aug, bounding_box_aug = augmentor(image=image, 
                                                bounding_boxes=bounding_box)
        
        # Discard the the bounding box going out the image completely   
        bounding_box_aug = bounding_box_aug.remove_out_of_image()

        # Clip the bounding box that are only partially out of th image
        bounding_box_aug = bounding_box_aug.clip_out_of_image()

        # Get rid of the the image if bounding box was discarded  
        if re.findall("Image...", str(bounding_box_aug)) == ["Image([]"]:
            pass
        
        else:
        
          # Create the augmented image file
          imageio.imwrite(augmented_path, image_aug) 

          # Update the image width and height after augmentation
          info_df = single_image.drop(["xmin", "ymin", "xmax", "ymax"], axis=1)    
          for index, _ in info_df.iterrows():
              info_df.at[index, "width"] = image_aug.shape[1]
              info_df.at[index, "height"] = image_aug.shape[0]

          # Add the prefix to each image to differentiate if required
          info_df["filename"] = info_df["filename"].apply(lambda x: x + image_postfix + ".jpg")

          # Create the augmented bounding boxes dataframe 
          bounding_box_df = bounding_boxes_to_df(bounding_box_aug)

          # Concatenate the filenames, height, width and bounding boxes 
          aug_df = pd.concat([info_df, bounding_box_df], axis=1)

          # Add all the information to augmentations_df we initialized above
          augmentations_df = pd.concat([augmentations_df, aug_df])
              
              
      
    # Remove index
    augmentations_df = augmentations_df.reset_index()
    augmentations_df = augmentations_df.drop(["index"], axis=1)

    # Return the Dataframe
    pbar.finish()
    return augmentations_df




augmented_images_df = image_aug(labels_df, image_directory, "aug_images", 
                                image_augmentations)

augmented_images_df = augmented_images_df.sort_values("filename", ignore_index= True)
augmented_images_df.to_csv("aug.csv")

# Check Dataset Size
print("Our total dataset Size before the augmentations was: ", len(labels_df))
print("Our total dataset Size after the augmentations is: ", len(augmented_images_df))

Processing cats/Cats_Test1017.png:   0%|          | 12/11058 [00:04<1:17:28,  2.38it/s]